In [2]:
import random
import numpy as np
import openai
import pandas as pd
import os
import sys
import time
from sklearn.neighbors import NearestNeighbors
from scipy.sparse import csr_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import openai
# Add the path to the constants file to the system path
sys.path.append('../../../')
from constants import *
from evaluation_utils import *
from path_utils import *
from ChatCompletion_OpenAI_API import *
from CF_utils import *

# OpenAI API Key
openai.api_key = OPENAI_API_KEY

# source code folder path
rec_sys_dir = get_rec_sys_directory()
print(f"Rec-sys directory: {rec_sys_dir}")

# data folder path
DATA_DIR = os.path.join(rec_sys_dir, '../data')
print(f"Data directory: {DATA_DIR}")

# data path
data_path = os.path.join(DATA_DIR, 'ml-1m/merged_data.dat')
print(f'Data path: {data_path}')

# output

CF_OUTPUT_PATH = os.path.join(DATA_DIR, 'ml-1m/output/CF_fewshot_output_path_ratings_per_user.dat')
print(f'Data path: {CF_OUTPUT_PATH}')

CF_RERUN_PATH = os.path.join(DATA_DIR, 'ml-1m/output/rerun_CF_fewshot_output_path_ratings_per_user.dat')
print(f'Data path: {CF_RERUN_PATH}')


# Constants for column names
USER_COLUMN_NAME = 'UserID'
TITLE_COLUMN_NAME = 'Title'
ITEM_ID_COLUMN = 'MovieID'
RATING_COLUMN_NAME = 'Rating'

SYSTEM_CONTENT = MOVIELENS_CONTENT_SYSTEM






Rec-sys directory: /Users/tnathu-ai/VSCode/recommender-system/recommender-system-openAI/code/notebook
Data directory: /Users/tnathu-ai/VSCode/recommender-system/recommender-system-openAI/code/notebook/../data
Data path: /Users/tnathu-ai/VSCode/recommender-system/recommender-system-openAI/code/notebook/../data/ml-1m/merged_data.dat
Data path: /Users/tnathu-ai/VSCode/recommender-system/recommender-system-openAI/code/notebook/../data/ml-1m/output/CF_fewshot_output_path_ratings_per_user.dat
Data path: /Users/tnathu-ai/VSCode/recommender-system/recommender-system-openAI/code/notebook/../data/ml-1m/output/rerun_CF_fewshot_output_path_ratings_per_user.dat


In [3]:
data = pd.read_csv(data_path)
data.head(3)

,UserID,MovieID,Rating,Timestamp,Gender,Age,Occupation,Zip-code,Title,Genres
0,1,1193,5,978300760,F,1,10,48067,One Flew Over the Cuckoo's Nest (1975),Drama
1,2,1193,5,978298413,M,56,16,70072,One Flew Over the Cuckoo's Nest (1975),Drama
2,12,1193,4,978220179,M,25,12,32793,One Flew Over the Cuckoo's Nest (1975),Drama


In [4]:
# data= data.head(50)  # Adjust as needed


In [5]:
# Create User-Item Interaction Matrix
interaction_matrix = pd.pivot_table(data, index=USER_COLUMN_NAME, columns=ITEM_ID_COLUMN, values=RATING_COLUMN_NAME).fillna(0)
csr_interaction_matrix = csr_matrix(interaction_matrix.values)

interaction_matrix

MovieID,1,2,3,4,5,6,7,8,9,10,...,3943,3944,3945,3946,3947,3948,3949,3950,3951,3952
UserID,,,,,,,,,,,,,,,,,,,,,
1,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6036,0.0,0.0,0.0,2.0,0.0,3.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6037,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6038,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
# Compute the user-user Pearson Correlation Coefficient Matrix
user_pcc_matrix = pearson_correlation(csr_interaction_matrix)
print(f'User PCC Matrix:\n{user_pcc_matrix}\n')
# Compute the item-item Pearson Correlation Coefficient Matrix
item_pcc_matrix = item_pearson_correlation(csr_interaction_matrix)
print(f'Item PCC Matrix:\n{item_pcc_matrix}\n')

User PCC Matrix:
[[ 1.          0.41666667 -0.33218192 ...  0.          0.05685735
  -0.04351941]
 [ 0.41666667  1.          0.23683386 ... -0.5         0.57207755
  -0.0271435 ]
 [-0.33218192  0.23683386  1.         ...  0.5         0.30927686
  -0.39528471]
 ...
 [ 0.         -0.5         0.5        ...  1.          0.27116307
  -0.39712226]
 [ 0.05685735  0.57207755  0.30927686 ...  0.27116307  1.
   0.24230884]
 [-0.04351941 -0.0271435  -0.39528471 ... -0.39712226  0.24230884
   1.        ]]



In [ ]:
# %%time

# Call the prediction function
results_df = predict_ratings_with_CF_item_PCC_and_save(
    data=data, 
    user_pcc_matrix=user_pcc_matrix, 
    item_pcc_matrix=item_pcc_matrix,
    user_column_name=USER_COLUMN_NAME, 
    movie_column_name=TITLE_COLUMN_NAME, 
    movie_id_column=ITEM_ID_COLUMN,
    rating_column_name=RATING_COLUMN_NAME, 
    num_ratings_per_user=NUM_RATINGS_PER_USER, 
    num_similar_users=NUM_SIMILAR_USERS,
    num_main_user_ratings=NUM_MAIN_USER_RATINGS,
    save_path=CF_OUTPUT_PATH, 
    seed=RANDOM_STATE
)



NameError: name 'predict_ratings_with_CF_item_PCC_and_save' is not defined

In [ ]:
# Evaluate updated CF model predictions
evaluate_model_predictions_rmse_mae(
    data_path=CF_OUTPUT_PATH,
    num_examples=NUM_EXAMPLES,
    actual_ratings_column='actual_rating',
    predicted_ratings_column='predicted_rating'
)

# First Iteration

In [ ]:
%%time


cf_predictions = predict_ratings_with_CF_item_PCC_and_save(data, 
                                                                       pcc_matrix, 
                                                                       save_path=CF_OUTPUT_PATH,
                                                                       user_column_name=USER_COLUMN_NAME,
                                                                       movie_column_name=TITLE_COLUMN_NAME,
                                                                       movie_id_column=ITEM_ID_COLUMN,
                                                                       rating_column_name=RATING_COLUMN_NAME, 
                                                                       num_ratings_per_user=NUM_RATINGS_PER_USER,
                                                                       num_main_user_ratings=NUM_MAIN_USER_RATINGS,
                                                                       num_similar_users=NUM_SIMILAR_USERS,)


In [ ]:
# Read the data
saved_data = pd.read_csv(CF_OUTPUT_PATH)

# Display the original data types
print("Original Data Types:")
print(saved_data.dtypes)
print("\n")

# Attempt to convert ratings to float and add a flag for conversion failure
saved_data['is_rating_float'] = pd.to_numeric(saved_data['predicted_rating'], errors='coerce').notna()

# Filter rows where ratings are not float
non_float_ratings = saved_data[saved_data['is_rating_float'] == False]

# total number of rows with non-float ratings
print(f"Total number of rows with non-float ratings: {len(non_float_ratings)}")

# rerun indices for non-float ratings
rerun_indices = non_float_ratings.index.tolist()
print(f"Rerun indices: {rerun_indices}")

# Display rows with non-float ratings
print("Rows with non-float ratings:")
non_float_ratings.head(3)


In [ ]:
%%time


rerun_failed_CF_fewshot_predictions(data, 
                                    pcc_matrix, 
                                    save_path=CF_OUTPUT_PATH,
                                    user_column_name=USER_COLUMN_NAME,
                                    movie_column_name=TITLE_COLUMN_NAME,
                                    movie_id_column=ITEM_ID_COLUMN,
                                    rating_column_name=RATING_COLUMN_NAME, 
                                    num_ratings_per_user=NUM_RATINGS_PER_USER,
                                    num_main_user_ratings=NUM_MAIN_USER_RATINGS,
                                    num_similar_users=NUM_SIMILAR_USERS, 
                                    new_path=CF_RERUN_PATH,
                                    rerun_indices=rerun_indices)


In [ ]:
rerun_data = pd.read_csv(CF_RERUN_PATH)
rerun_data.info()

In [ ]:
CF_RERUN_PATH

In [ ]:
# Evaluate updated CF model predictions
evaluate_model_predictions_rmse_mae(
    data_path=CF_RERUN_PATH,
    num_examples=NUM_EXAMPLES,
    actual_ratings_column='actual_rating',
    predicted_ratings_column='predicted_rating'
)

# Second iteration

In [ ]:
%%time
# output
CF_OUTPUT_PATH = os.path.join(DATA_DIR, 'ml-1m/output/CF_fewshot_output_path_ratings_per_user_2nd.dat')
print(f'Data path: {data_path}')

CF_RERUN_PATH = os.path.join(DATA_DIR, 'ml-1m/output/rerun_CF_fewshot_output_path_ratings_per_user_2nd.dat')
print(f'Data path: {data_path}')




cf_predictions = predict_ratings_with_CF_item_PCC_and_save(data, 
                                                                       pcc_matrix, 
                                                                       save_path=CF_OUTPUT_PATH,
                                                                       user_column_name=USER_COLUMN_NAME,
                                                                       movie_column_name=TITLE_COLUMN_NAME,
                                                                       movie_id_column=ITEM_ID_COLUMN,
                                                                       rating_column_name=RATING_COLUMN_NAME, 
                                                                       num_ratings_per_user=NUM_RATINGS_PER_USER,
                                                                       num_main_user_ratings=NUM_MAIN_USER_RATINGS,
                                                                       num_similar_users=NUM_SIMILAR_USERS, )


In [ ]:
# Read the data
saved_data = pd.read_csv(CF_OUTPUT_PATH)

# Display the original data types
print("Original Data Types:")
print(saved_data.dtypes)
print("\n")

# Attempt to convert ratings to float and add a flag for conversion failure
saved_data['is_rating_float'] = pd.to_numeric(saved_data['predicted_rating'], errors='coerce').notna()

# Filter rows where ratings are not float
non_float_ratings = saved_data[saved_data['is_rating_float'] == False]

# total number of rows with non-float ratings
print(f"Total number of rows with non-float ratings: {len(non_float_ratings)}")

# rerun indices for non-float ratings
rerun_indices = non_float_ratings.index.tolist()
print(f"Rerun indices: {rerun_indices}")

# Display rows with non-float ratings
print("Rows with non-float ratings:")
non_float_ratings.head(3)


In [ ]:
%%time


rerun_failed_CF_fewshot_predictions(data, 
                                    pcc_matrix, 
                                    save_path=CF_OUTPUT_PATH,
                                    user_column_name=USER_COLUMN_NAME,
                                    movie_column_name=TITLE_COLUMN_NAME,
                                    movie_id_column=ITEM_ID_COLUMN,
                                    rating_column_name=RATING_COLUMN_NAME, 
                                    num_ratings_per_user=NUM_RATINGS_PER_USER,
                                    num_main_user_ratings=NUM_MAIN_USER_RATINGS,
                                    num_similar_users=NUM_SIMILAR_USERS, , 
                                    new_path=CF_RERUN_PATH,
                                    rerun_indices=rerun_indices)


# Evaluate updated CF model predictions
evaluate_model_predictions_rmse_mae(
    data_path=CF_RERUN_PATH,
    num_examples=NUM_EXAMPLES,
    actual_ratings_column='actual_rating',
    predicted_ratings_column='predicted_rating'
)


# Third iteration

In [ ]:
%%time
# output
CF_OUTPUT_PATH = os.path.join(DATA_DIR, 'ml-1m/output/CF_fewshot_output_path_ratings_per_user_3rd.dat')
print(f'Data path: {data_path}')

CF_RERUN_PATH = os.path.join(DATA_DIR, 'ml-1m/output/rerun_CF_fewshot_output_path_ratings_per_user_3rd.dat')
print(f'Data path: {data_path}')




cf_predictions = predict_ratings_with_CF_item_PCC_and_save(data, 
                                                                       pcc_matrix, 
                                                                       save_path=CF_OUTPUT_PATH,
                                                                       user_column_name=USER_COLUMN_NAME,
                                                                       movie_column_name=TITLE_COLUMN_NAME,
                                                                       movie_id_column=ITEM_ID_COLUMN,
                                                                       rating_column_name=RATING_COLUMN_NAME, 
                                                                       num_ratings_per_user=NUM_RATINGS_PER_USER,
                                                                       num_main_user_ratings=NUM_MAIN_USER_RATINGS,
                                                                       num_similar_users=NUM_SIMILAR_USERS, )


In [ ]:
# Read the data
saved_data = pd.read_csv(CF_OUTPUT_PATH)

# Display the original data types
print("Original Data Types:")
print(saved_data.dtypes)
print("\n")

# Attempt to convert ratings to float and add a flag for conversion failure
saved_data['is_rating_float'] = pd.to_numeric(saved_data['predicted_rating'], errors='coerce').notna()

# Filter rows where ratings are not float
non_float_ratings = saved_data[saved_data['is_rating_float'] == False]

# total number of rows with non-float ratings
print(f"Total number of rows with non-float ratings: {len(non_float_ratings)}")

# rerun indices for non-float ratings
rerun_indices = non_float_ratings.index.tolist()
print(f"Rerun indices: {rerun_indices}")

# Display rows with non-float ratings
print("Rows with non-float ratings:")
non_float_ratings.head(3)


In [ ]:
%%time


rerun_failed_CF_fewshot_predictions(data, 
                                    pcc_matrix, 
                                    save_path=CF_OUTPUT_PATH,
                                    user_column_name=USER_COLUMN_NAME,
                                    movie_column_name=TITLE_COLUMN_NAME,
                                    movie_id_column=ITEM_ID_COLUMN,
                                    rating_column_name=RATING_COLUMN_NAME, 
                                    num_ratings_per_user=NUM_RATINGS_PER_USER,
                                    num_main_user_ratings=NUM_MAIN_USER_RATINGS,
                                    num_similar_users=NUM_SIMILAR_USERS, , 
                                    new_path=CF_RERUN_PATH,
                                    rerun_indices=rerun_indices)


# Evaluate updated CF model predictions
evaluate_model_predictions_rmse_mae(
    data_path=CF_RERUN_PATH,
    num_examples=NUM_EXAMPLES,
    actual_ratings_column='actual_rating',
    predicted_ratings_column='predicted_rating'
)


# Fourth iteration

In [ ]:
%%time
# output
CF_OUTPUT_PATH = os.path.join(DATA_DIR, 'ml-1m/output/CF_fewshot_output_path_ratings_per_user_4th.dat')
print(f'Data path: {data_path}')

CF_RERUN_PATH = os.path.join(DATA_DIR, 'ml-1m/output/rerun_CF_fewshot_output_path_ratings_per_user_4th.dat')
print(f'Data path: {data_path}')




cf_predictions = predict_ratings_with_CF_item_PCC_and_save(data, 
                                                                       pcc_matrix, 
                                                                       save_path=CF_OUTPUT_PATH,
                                                                       user_column_name=USER_COLUMN_NAME,
                                                                       movie_column_name=TITLE_COLUMN_NAME,
                                                                       movie_id_column=ITEM_ID_COLUMN,
                                                                       rating_column_name=RATING_COLUMN_NAME, 
                                                                       num_ratings_per_user=NUM_RATINGS_PER_USER,
                                                                       num_main_user_ratings=NUM_MAIN_USER_RATINGS,
                                                                       num_similar_users=NUM_SIMILAR_USERS, )


In [ ]:
# Read the data
saved_data = pd.read_csv(CF_OUTPUT_PATH)

# Display the original data types
print("Original Data Types:")
print(saved_data.dtypes)
print("\n")

# Attempt to convert ratings to float and add a flag for conversion failure
saved_data['is_rating_float'] = pd.to_numeric(saved_data['predicted_rating'], errors='coerce').notna()

# Filter rows where ratings are not float
non_float_ratings = saved_data[saved_data['is_rating_float'] == False]

# total number of rows with non-float ratings
print(f"Total number of rows with non-float ratings: {len(non_float_ratings)}")

# rerun indices for non-float ratings
rerun_indices = non_float_ratings.index.tolist()
print(f"Rerun indices: {rerun_indices}")

# Display rows with non-float ratings
print("Rows with non-float ratings:")
non_float_ratings.head(3)


In [ ]:
%%time


rerun_failed_CF_fewshot_predictions(data, 
                                    pcc_matrix, 
                                    save_path=CF_OUTPUT_PATH,
                                    user_column_name=USER_COLUMN_NAME,
                                    movie_column_name=TITLE_COLUMN_NAME,
                                    movie_id_column=ITEM_ID_COLUMN,
                                    rating_column_name=RATING_COLUMN_NAME, 
                                    num_ratings_per_user=NUM_RATINGS_PER_USER,
                                    num_main_user_ratings=NUM_MAIN_USER_RATINGS,
                                    num_similar_users=NUM_SIMILAR_USERS,
                                    new_path=CF_RERUN_PATH,
                                    rerun_indices=rerun_indices)


# Evaluate updated CF model predictions
evaluate_model_predictions_rmse_mae(
    data_path=CF_RERUN_PATH,
    num_examples=NUM_EXAMPLES,
    actual_ratings_column='actual_rating',
    predicted_ratings_column='predicted_rating'
)


# Fifth Iteration

In [ ]:
%%time
# output
CF_OUTPUT_PATH = os.path.join(DATA_DIR, 'ml-1m/output/CF_fewshot_output_path_ratings_per_user_5th.dat')
print(f'Data path: {data_path}')

CF_RERUN_PATH = os.path.join(DATA_DIR, 'ml-1m/output/rerun_CF_fewshot_output_path_ratings_per_user_5th.dat')
print(f'Data path: {data_path}')




cf_predictions = predict_ratings_with_CF_item_PCC_and_save(data, 
                                                                       pcc_matrix, 
                                                                       save_path=CF_OUTPUT_PATH,
                                                                       user_column_name=USER_COLUMN_NAME,
                                                                       movie_column_name=TITLE_COLUMN_NAME,
                                                                       movie_id_column=ITEM_ID_COLUMN,
                                                                       rating_column_name=RATING_COLUMN_NAME, 
                                                                       num_ratings_per_user=NUM_RATINGS_PER_USER,
                                                                       num_main_user_ratings=NUM_MAIN_USER_RATINGS,
                                                                       num_similar_users=NUM_SIMILAR_USERS, )


In [ ]:
# Read the data
saved_data = pd.read_csv(CF_OUTPUT_PATH)

# Display the original data types
print("Original Data Types:")
print(saved_data.dtypes)
print("\n")

# Attempt to convert ratings to float and add a flag for conversion failure
saved_data['is_rating_float'] = pd.to_numeric(saved_data['predicted_rating'], errors='coerce').notna()

# Filter rows where ratings are not float
non_float_ratings = saved_data[saved_data['is_rating_float'] == False]

# total number of rows with non-float ratings
print(f"Total number of rows with non-float ratings: {len(non_float_ratings)}")

# rerun indices for non-float ratings
rerun_indices = non_float_ratings.index.tolist()
print(f"Rerun indices: {rerun_indices}")

# Display rows with non-float ratings
print("Rows with non-float ratings:")
non_float_ratings.head(3)


In [ ]:
%%time


rerun_failed_CF_fewshot_predictions(data, 
                                    pcc_matrix, 
                                    save_path=CF_OUTPUT_PATH,
                                    user_column_name=USER_COLUMN_NAME,
                                    movie_column_name=TITLE_COLUMN_NAME,
                                    movie_id_column=ITEM_ID_COLUMN,
                                    rating_column_name=RATING_COLUMN_NAME, 
                                    num_ratings_per_user=NUM_RATINGS_PER_USER,
                                    num_main_user_ratings=NUM_MAIN_USER_RATINGS,
                                    num_similar_users=NUM_SIMILAR_USERS,
                                    new_path=CF_RERUN_PATH,
                                    rerun_indices=rerun_indices)


# Evaluate updated CF model predictions
evaluate_model_predictions_rmse_mae(
    data_path=CF_RERUN_PATH,
    num_examples=NUM_EXAMPLES,
    actual_ratings_column='actual_rating',
    predicted_ratings_column='predicted_rating'
)


# Sixth Iteration

In [ ]:
%%time
# output
CF_OUTPUT_PATH = os.path.join(DATA_DIR, 'ml-1m/output/CF_fewshot_output_path_ratings_per_user_6th.dat')
print(f'Data path: {data_path}')

CF_RERUN_PATH = os.path.join(DATA_DIR, 'ml-1m/output/rerun_CF_fewshot_output_path_ratings_per_user_6th.dat')
print(f'Data path: {data_path}')


In [ ]:
%%time
# output
CF_OUTPUT_PATH = os.path.join(DATA_DIR, 'ml-1m/output/CF_fewshot_output_path_ratings_per_user_6th.dat')
print(f'Data path: {data_path}')

CF_RERUN_PATH = os.path.join(DATA_DIR, 'ml-1m/output/rerun_CF_fewshot_output_path_ratings_per_user_6th.dat')
print(f'Data path: {data_path}')




cf_predictions = predict_ratings_with_CF_item_PCC_and_save(data, 
                                                                       pcc_matrix, 
                                                                       save_path=CF_OUTPUT_PATH,
                                                                       user_column_name=USER_COLUMN_NAME,
                                                                       movie_column_name=TITLE_COLUMN_NAME,
                                                                       movie_id_column=ITEM_ID_COLUMN,
                                                                       rating_column_name=RATING_COLUMN_NAME, 
                                                                       num_ratings_per_user=NUM_RATINGS_PER_USER,
                                                                       num_main_user_ratings=NUM_MAIN_USER_RATINGS,
                                                                       num_similar_users=NUM_SIMILAR_USERS, )


In [ ]:
# Read the data
saved_data = pd.read_csv(CF_OUTPUT_PATH)

# Display the original data types
print("Original Data Types:")
print(saved_data.dtypes)
print("\n")

# Attempt to convert ratings to float and add a flag for conversion failure
saved_data['is_rating_float'] = pd.to_numeric(saved_data['predicted_rating'], errors='coerce').notna()

# Filter rows where ratings are not float
non_float_ratings = saved_data[saved_data['is_rating_float'] == False]

# total number of rows with non-float ratings
print(f"Total number of rows with non-float ratings: {len(non_float_ratings)}")

# rerun indices for non-float ratings
rerun_indices = non_float_ratings.index.tolist()
print(f"Rerun indices: {rerun_indices}")

# Display rows with non-float ratings
print("Rows with non-float ratings:")
non_float_ratings.head(3)


In [ ]:
%%time


rerun_failed_CF_fewshot_predictions(data, 
                                    pcc_matrix, 
                                    save_path=CF_OUTPUT_PATH,
                                    user_column_name=USER_COLUMN_NAME,
                                    movie_column_name=TITLE_COLUMN_NAME,
                                    movie_id_column=ITEM_ID_COLUMN,
                                    rating_column_name=RATING_COLUMN_NAME, 
                                    num_ratings_per_user=NUM_RATINGS_PER_USER,
                                    num_main_user_ratings=NUM_MAIN_USER_RATINGS,
                                    num_similar_users=NUM_SIMILAR_USERS,
                                    new_path=CF_RERUN_PATH,
                                    rerun_indices=rerun_indices)


# Evaluate updated CF model predictions
evaluate_model_predictions_rmse_mae(
    data_path=CF_RERUN_PATH,
    num_examples=NUM_EXAMPLES,
    actual_ratings_column='actual_rating',
    predicted_ratings_column='predicted_rating'
)
